In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function buildPredsummary(summary, chosenactions, inputsentences, select_index)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(summary, chosenactions, inputsentences, select_index)
    n = inputsentences:size(1)
    k = inputsentences:size(2)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(sentences[1]:double())
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SELECT = 2
SKIP = 1

In [4]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)                   
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2) 
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [68]:
local SKIP = 1
local SELECT = 2
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)
-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end
-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)                   
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2) 
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end
model = buildModel('bow', b, embDim, 'f1', false, false)
preds = model:forward({sentences[1], queries, torch.zeros(n, q)})
print("predictions = ")
print(preds)
-- Pulling the best actions
qMax, qindx = torch.max(preds, 2)
-- Here's the fast way to select the optimal action for each query
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

Running bag-of-words model to learn f1	


predictions = 	
-0.1286 -0.0033
 0.0443  0.1099
-0.1623  0.1136
 0.0705  0.0436
 0.0356 -0.0867
-0.1926  0.1419
-0.0903 -0.2053
 0.0234 -0.1694
-0.0477  0.2595
-0.0740 -0.0104
[torch.DoubleTensor of size 10x2]



In [69]:
n = sentences[1]:size(1)
k = sentences[1]:size(2)

In [70]:
sentences[1]:double(), actions

   5   64   97   36   91   22   24
  81   90   38   13   71   83   44
  86   49   98   25    7   81   34
  92    5   16   84   26   20    7
  14  100   41   20   32   19   38
   8   65   11   78   87   31   39
  92   69   23   44   57   16   93
  16   36    9   35   40   17   20
  81   80   15   76    5   26   80
  67   88   88   93   15   64   95
[torch.DoubleTensor of size 10x7]

 0  1
 0  1
 0  1
 1  0
 1  0
 0  1
 1  0
 1  0
 0  1
 0  1
[torch.DoubleTensor of size 10x2]



In [71]:
actionmatrix = actions:select(2, 2):resize(1, n):view(n, 1):expand(n, k):clone()

In [72]:
actionmatrix

 1  1  1  1  1  1  1
 0  0  0  0  0  0  0
 1  1  1  1  1  1  1
 0  0  0  0  0  0  0
 1  1  1  1  1  1  1
 1  1  1  1  1  1  1
 0  0  0  0  0  0  0
 1  1  1  1  1  1  1
 0  0  0  0  0  0  0
 0  0  0  0  0  0  0
[torch.DoubleTensor of size 10x7]



In [73]:
actionmatrix:cmul(sentences[1]:double())

   5   64   97   36   91   22   24
   0    0    0    0    0    0    0
  86   49   98   25    7   81   34
   0    0    0    0    0    0    0
  14  100   41   20   32   19   38
   8   65   11   78   87   31   39
   0    0    0    0    0    0    0
  16   36    9   35   40   17   20
   0    0    0    0    0    0    0
   0    0    0    0    0    0    0
[torch.DoubleTensor of size 10x7]



In [52]:
:cmul(sentences[1]:double())

Columns 1 to 6
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 2.6234e+19  2.6234e+19  2.6234e+19  2.6234e+19  2.6234e+19  2.6234e+19
 7.5638e+23  7.5638e+23  7.5638e+23  7.5638e+23  7.5638e+23  7.5638e+23
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 1.4257e+24  1.4257e+24  1.4257e+24  1.4257e+24  1.4257e+24  1.4257e+24
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 1.1699e+24  1.1699e+24  1.1699e+24  1.1699e+24  1.1699e+24  1.1699e+24
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 4.5226e+19  4.5226e+19  4.5226e+19  4.5226e+19  4.5226e+19  4.5226e+19

Columns 7 to 7
 0.0000e+00
 0.0000e+00
 2.6234e+19
 7.5638e+23
 0.0000e+00
 1.4257e+24
 0.0000e+00
 1.1699e+24
 0.0000e+00
 4.5226e+19
[torch.DoubleTensor of size 10x7]



In [10]:
actions:select(2,2):cum(sentences)

[string "local f = function() return actions:select(2,..."]:1: attempt to call method 'cum' (a nil value)
stack traceback:
	[string "local f = function() return actions:select(2,..."]:1: in function 'f'
	[string "local f = function() return actions:select(2,..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...vierarceo/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x01034e0d20: 

In [6]:
-- Here's the slow way to build the predicted summary...
    -- This is what I'd like to optimize further, right now it's a for loop
    -- it'd be great if we could skip the loop...maybe with a matrix multiplication to 
    -- wipe out all of the non-zero elements...
predsummary = buildPredsummary(predsummary, actions, sentences[1], SELECT)
print("predicted summary = ")
print(predsummary)

predicted summary = 	
  0   0   0   0   0   0   0
 34  84  47  58   6  24  15
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 61  16  87  11  15  15  32
[torch.DoubleTensor of size 10x7]



In [14]:
actions:select(2, 1)

 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
[torch.DoubleTensor of size 10]



In [ ]:
function stackMemory(newinput, memory_hist, memsize, adapt, use_cuda)
    local sentMemory = torch.cat(newinput[1][1]:double(), memory_hist[1][1]:double(), 1)
    local queryMemory = torch.cat(newinput[1][2]:double(), memory_hist[1][2]:double(), 1)
    local sumryMemory = torch.cat(newinput[1][3]:double(), memory_hist[1][3]:double(), 1)
    local rewardMemory = torch.cat(newinput[2]:double(), memory_hist[2]:double(), 1)

    if adapt then
        regMemory = torch.cat(newinput[4]:double(), memory_hist[4]:double(), 1)
    end 

    if use_cuda then 
        actionMemory = torch.cat(newinput[3]:double(), memory_hist[3]:double(), 1)
    else 
        actionMemory = torch.cat(newinput[3], memory_hist[3], 1)
    end
    --- specifying rows to index 
    if sentMemory:size(1) <= memsize then
        nend = sentMemory:size(1)
        nstart = 1
    else 
        nstart = math.max(memsize - sentMemory:size(1), 1)
        nend = memsize + nstart
    end
    --- Selecting n last data points
    sentMemory = sentMemory[{{nstart, nend}}]
    queryMemory = queryMemory[{{nstart, nend}}]
    sumryMemory = sumryMemory[{{nstart, nend}}]
    rewardMemory = rewardMemory[{{nstart, nend}}]
    actionMemory = actionMemory[{{nstart, nend}}]

    if use_cuda then
        inputMemory = {sentMemory:cuda(), queryMemory:cuda(), sumryMemory:cuda()}
        rewardMemory = rewardMemory:cuda()
        actionMemory = torch.ByteTensor(#actionMemory):copy(actionMemory):cuda()
    end

    inputMemory = {sentMemory, queryMemory, sumryMemory}
    if adapt then
        regMemory = regMemory[{{nstart, nend}}]
        return {inputMemory, rewardMemory, actionMemory, regMemory}
    end 
    return {inputMemory, rewardMemory, actionMemory}
end    

In [ ]:
querySize = queries:size(2)
streamSize = sentences[1]:size(2)
-- summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
-- queryBatch = queries:view(1, querySize):expand(streamSize, querySize) 
-- input = {sentenceStream, queryBatch, summaryBatch}

In [ ]:
model = buildModel('bow', b, embDim, 'f1', false, false)

In [ ]:
preds = model:forward({sentences[1], queries, torch.zeros(n, q)})
print(preds)

In [ ]:
qMax, qindx = torch.max(preds, 2)
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

In [ ]:
predsummary = torch.zeros(sentences[1]:size())
for i=1, actions:size(1) do
    if actions[i][2]==1 then
        predsummary[i]:copy(sentences[1][i])
    end
end

In [ ]:
indices = torch.linspace(1, actions:size(1), actions:size(1) ):long()
selected = indices[actions:select(2,1):eq(1)]

In [ ]:
torch.totable(selected)

In [ ]:
predsummary = torch.zeros(sentences[1]:size())

In [ ]:
sentences[1]:index(1, torch.LongTensor(selected)):double()

In [ ]:
-- predsummary:scatter(1, torch.LongTensor(selected), sentences[1]:double())

In [ ]:
predsummary:index(1, torch.LongTensor(selected)):copy(sentences[1]:index(1, torch.LongTensor(selected)):double())

In [ ]:
torch.eq(actions:select(2,2), 1):view(n, 1):expand(n, k)

In [ ]:
predsummary

In [ ]:
test = torch.zeros(n, 7)
-- test:copy(actions:selec(2, 1))

In [ ]:
-- torch.zeros(n, 7):scatter(2, actions:select(2, 1), 

In [ ]:
maskedSelect = nn.MaskedSelect()
mask1 = torch.eq(actions:select(2,2), 1):view(n, 1):expand(n, k)
allTokens = sentences[1]:maskedSelect(mask1)
mask2 = torch.gt(allTokens,0)
allTokens = allTokens:maskedSelect(mask2)

In [ ]:
torch.zeros(n, 15):scatter(2, qindx, sentences[1]:double())

In [ ]:
sentences[1]

In [ ]:
indices = torch.linspace(1, x:size(1), x:size(1) ):long()
selected = indices[x:eq(1)]

In [ ]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
-- sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)

In [ ]:
indx = torch.range(1, 10)
-- Summary index for selecting the words
sumindx = nn.MaskedSelect():forward({indx, actions:select(2,1):resize(10, 1):byte()})  --- sentences
-- need a LongTensor to do the selection
sumindx = torch.LongTensor(sumindx:size(1)):copy(sumindx)
--- This gives the wrong selection we want
print(sentences:index(1, sumindx))